# AutoGluon: PDF Document Classification

## Objective
This notebook demonstrates **PDF document classification** using AutoGluon. It classifies PDF documents into categories based on their content, structure, and visual layout.

## Use Case
PDF classification is useful for:
- Contract type identification (lease, employment, NDA, etc.)
- Research paper categorization by field
- Financial document sorting (statements, reports, invoices)
- Legal brief classification
- Technical manual categorization
- Resume screening and categorization

## Key Features
- Handles multi-page PDF documents
- Extracts text and visual features
- Considers document layout and structure
- Works with both text-based and scanned PDFs

In [ ]:
# Install AutoGluon
!pip install -q autogluon

In [ ]:
# Import libraries
import pandas as pd
import numpy as np
from autogluon.tabular import TabularDataset, TabularPredictor
import os
import shutil

In [ ]:
# Load dataset
# TODO: Upload your PDF dataset or use URL
# Dataset should have:
# - 'pdf' or 'file' column (paths to PDF files)
# - 'label' column (PDF category)
# - Optional: 'text' column (extracted text), 'pages' (number of pages)

# Example: train_data = TabularDataset('path/to/pdf_dataset.csv')

train_data = None  # Replace with your data
test_data = None   # Replace with your data

print("Dataset loaded successfully!")
if train_data is not None:
    print(f"Training data shape: {train_data.shape}")
    print(train_data.head())

In [ ]:
# Set label column
LABEL = 'label'  # TODO: Replace with your label column name (e.g., 'document_type', 'category')

In [ ]:
# Auto-detect problem type based on label
if train_data is not None and LABEL in train_data.columns:
    # Check if the label is numeric (regression) or categorical (classification)
    if pd.api.types.is_numeric_dtype(train_data[LABEL]):
        # Check if it's continuous or discrete
        unique_ratio = train_data[LABEL].nunique() / len(train_data)
        if unique_ratio > 0.05:  # More than 5% unique values suggests regression
            problem_type = 'regression'
            eval_metric = 'rmse'
        else:
            problem_type = 'classification'
            eval_metric = 'roc_auc'
    else:
        problem_type = 'classification'
        eval_metric = 'roc_auc'
else:
    # Default to classification for PDF tasks
    problem_type = 'classification'
    eval_metric = 'roc_auc'

print(f"Problem Type: {problem_type}")
print(f"Evaluation Metric: {eval_metric}")
print("\nNote: PDF classification handles multi-page documents with text and images.")

In [ ]:
# Train the model
# AutoGluon will automatically extract features from PDFs
predictor = TabularPredictor(
    label=LABEL,
    problem_type=problem_type,
    eval_metric=eval_metric,
    path='./autogluon-pdf-model'
).fit(
    train_data=train_data,
    presets='medium_quality',
    time_limit=900
)

print("Model training completed!")

In [ ]:
# Display and save leaderboard
leaderboard = predictor.leaderboard(test_data, silent=True)
print("\nModel Leaderboard:")
print(leaderboard)

# Save leaderboard to CSV
leaderboard.to_csv('leaderboard.csv', index=False)
print("\nLeaderboard saved to leaderboard.csv")

In [ ]:
# Display and save feature importance
try:
    feature_importance = predictor.feature_importance(test_data)
    print("\nFeature Importance:")
    print(feature_importance)
    
    # Save feature importance to CSV
    feature_importance.to_csv('feature_importance.csv')
    print("\nFeature importance saved to feature_importance.csv")
except Exception as e:
    print(f"Could not compute feature importance: {e}")

In [ ]:
# Make predictions
if test_data is not None:
    predictions = predictor.predict(test_data)
    print("\nPredictions (PDF Categories):")
    print(predictions.head())
    
    # For classification, also show prediction probabilities
    if problem_type == 'classification':
        pred_probs = predictor.predict_proba(test_data)
        print("\nPrediction Probabilities:")
        print(pred_probs.head())
        
    # Example: Classify new PDF documents
    print("\nExample usage for new PDFs:")
    print("new_pdfs = pd.DataFrame({'pdf': ['path/to/contract.pdf']})")
    print("prediction = predictor.predict(new_pdfs)")
    print("\nThe model works with:")
    print("- Single-page and multi-page PDFs")
    print("- Text-based and scanned PDFs")
    print("- Various PDF formats and structures")

In [ ]:
# Save model artifacts as zip file
model_path = './autogluon-pdf-model'
zip_filename = 'autogluon_pdf_model'

if os.path.exists(model_path):
    shutil.make_archive(zip_filename, 'zip', model_path)
    print(f"\nModel artifacts saved to {zip_filename}.zip")
else:
    print("Model path not found. Train the model first.")